In [3]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from utils.cuda_cluster import *
from utils.dataset import read_data, factorize_small_cardinality
from utils.util import get_day_phase, get_hours_from_midnight_of_current_day

import core.config as conf

In [4]:
client

Client Scheduler: tcp://127.0.0.1:39821 Dashboard: http://127.0.0.1:35701/status,Cluster Workers: 1 Cores: 1 Memory: 33.47 GB


## 1. Load data

In [5]:
# data_path = conf.raw_data_path + '*' # for all dataset
data_path = conf.raw_data_path + 'part-00175'
ori_df = read_data(data_path)

number of rows: 3033347


In [6]:
for col in ["tweet_id", "engaged_with_user_id", "enaging_user_id"]:
    ori_df, _ = factorize_small_cardinality(ori_df, col)
    

In [13]:
df = ori_df[["tweet_id_encode", "engaged_with_user_id_encode", "enaging_user_id_encode"]]

In [16]:
df.head()

,tweet_id_encode,engaged_with_user_id_encode,enaging_user_id_encode
id,,,
1,2214622,1148208,553952
2,2136815,1490149,1966134
3,1986297,547972,589453
4,35685,1360160,1247846
5,1465500,1012520,1870103


In [17]:
df.info()

<class 'dask_cudf.core.DataFrame'>
Columns: 3 entries, tweet_id_encode to enaging_user_id_encode
dtypes: int64(3)